# Machine Learning Algorithms for Ionic Conductivity of LLZO-type Garnets

### Authorship and credits

<b> nanoHUB tools by: </b>  <i>Juan Carlos Verduzco</i> and <i>Alejandro Strachan</i>, Materials Engineering, Purdue University <br>
<b> Database curated by: </b> <i>Juan Carlos Verduzco</i>, Materials Engineering, Purdue University <br>


## Overview

In this notebook, we explore regularization on our Neural Network model. This relates to our publication: "An active learning approach for the design of doped LLZO ceramic garnets for battery applications" (Submitted)

<br>

**Outline:**

1. Querying / Processing Data <br>
2. Obtaining features/descriptors from Matminer <br>
3. Regression Models (Architecture 1) <br>
    3.1 Dropuout <br>
    3.2 L2 Regularization <br>
    3.3 L1 Regularization  <br>
4. Regression Models (Architecture 2) <br>
    4.1 Dropuout <br>
    4.2 L2 Regularization <br>
    4.3 L1 Regularization  <br>


Notes: This notebook uses tools from [Citrination](https://citrination.com/) and requires an account with an API key.

## Libraries

This notebook requires several libraries to be installed. They are separated in blocks depending on their usage.

In [1]:
# PLOTTING (MATPLOTLIB)
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from lolopy.learners import RandomForestRegressor

# PYTHON

import pandas as pd
import numpy as np
import os
import sys
import random

# MACHINE LEARNING
import tensorflow as tf
from tensorflow import keras
from keras import initializers, regularizers
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
print(keras.__version__)

# CITRINATION / MATMINER

from matminer.data_retrieval.retrieve_Citrine import CitrineDataRetrieval
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from sklearn.model_selection import KFold
from pymatgen import Composition
from scipy.stats import norm

# PLOTTING (PLOTLY)
import plotly 
import plotly.graph_objs as go
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)


# This snipped refers to the adding of the CitrineKey on the main page of the tool. If you are running this notebook by itself, please comment it out and write your citrinekey in the cell below.
file = open(os.path.expanduser('~/.citrinetools.txt'),"r+")
apikey = file.readline()
file.close()

Using TensorFlow backend.


2.2.4-tf


---
## 1. Querying a Database from Citrination

Matminer offers API tools to facilitate querying of databases like the Materials Project and Citrination. An individual **Citrine Key** is required for the query command <i>CitrineDataRetrieval</i>.

Data is stored in a Pandas Dataframe and the list of possible properties to be queried can be consulted by setting the print_properties_options parameter to **True**.

In [2]:
cdr = CitrineDataRetrieval(apikey) # Citrine Key

data = cdr.get_dataframe(criteria={'data_set_id': 184812}, print_properties_options=False) # LLZO Database
property_interest = 'Ionic Conductivity' # Property to be queried

display(data.head(n=10))

  0%|          | 0/189 [00:00<?, ?it/s]/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


100%|██████████| 189/189 [00:04<00:00, 42.07it/s]


,chemicalFormula,preparation,references,Crystallographic Structure,Ionic Conductivity,Ionic Conductivity-units,Measuring Temperature,Sintering Temperature,Sintering Temperature-units,Space Group,Year Published
1,Li6.1La3Zr2Ga0.3O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1021/acsami.6b13902', 'doi':...",Cubic,11.2,10^-4 S/cm,25,1100,$\circ$C,Ia3d,2017
2,Li7La2Ta2Ba1O12.5,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1007/s00339-008-4494-2', 'do...",Cubic,1.01,10^-4 S/cm,50,900,$\circ$C,Ia3d,2008
3,Li6La2Nb2Ba1O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1149/1.2800764', 'doi': '10....",Cubic,0.155,10^-4 S/cm,17,950,$\circ$C,NaN,2008
4,Li6.4La3Zr2Ga0.15Al0.05O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1021/acs.chemmater.6b00579',...",Cubic,8.8,10^-4 S/cm,20,1230,$\circ$C,I43d,2016
5,Li7La3Zr2O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1021/acs.inorgchem.5b00184',...",Cubic,1.74,10^-4 S/cm,25,900,$\circ$C,Ia3d,2015
6,Li6.4La3Zr2Ga0.2O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1021/cm5008069', 'doi': '10....",Cubic,9,10^-4 S/cm,24,1085,$\circ$C,NaN,2014
7,Li6.7La3Zr2Ga0.1O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1021/acsami.6b13902', 'doi':...",Mixed Cubic / Tetragonal,0.25,10^-4 S/cm,25,1100,$\circ$C,NaN,2017
8,Li6La2Ta1Nb1Ba1O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1007/s11581-013-0863-8', 'do...",Cubic,0.0309,10^-4 S/cm,25,900,$\circ$C,NaN,2013
9,Li5.2La2Nb1.9Y0.1O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1021/jp304737x', 'doi': '10....",Cubic,0.2,10^-4 S/cm,25,900,$\circ$C,Ia3d,2012
10,Li7.06La3Zr1.94Y0.06O12,"[{'name': 'SINTERING', 'details': [{'name': 'S...","[{'citation': '10.1149/2.088308jes', 'doi': '1...",Cubic,0.01,10^-4 S/cm,25,950,$\circ$C,Ia3d,2013


This is a utility function that will transform the <i>chemicalFormula</i> column into a Matminer composition object, which will be then used to extract features.

In [3]:
def get_composition(c): # Function to get compositions from chemical formula using pymatgen
    try:
        return Composition(c)
    except:
        return None

We will use the utility function to transform the <i>chemicalFormula</i> column, and we'll typecast relevant columns into numeric types.
<br>
For this specific application, we'll introduce some filters for the dataframe. We are interested in measurements in structures that are cubic and measured at room temperature (Defined as 18°C < T < 30°C)

In [4]:
data['composition'] = data['chemicalFormula'].apply(get_composition) # Transformation of chemicalformula string into Matminer composition
data['Measuring Temperature'] = pd.to_numeric(data['Measuring Temperature'], errors='coerce') # Transformation of Measuring Temp dataframe column from type <str> to a numberic type <int>
data[property_interest] = pd.to_numeric(data[property_interest], errors='coerce') # Transformation of our property of interest dataframe column from type <str> to a numberic type <int>
data["Year Published"] = pd.to_numeric(data["Year Published"], errors='coerce') # Transformation of our property of interest dataframe column from type <str> to a numberic type <int>

data = data[data['Crystallographic Structure'] == 'Cubic'] # Filter all non-cubic structures
data = data[data['Measuring Temperature']<30] # Filter all high temperature measurements (over room temperature)
data = data[data['Measuring Temperature']>18] # Filter all low temperature measurements (over room temperature)

data.reset_index(drop=True, inplace=True) # Reindexing of dataframe rows

Before removing duplicates, we will store all the experimental values for compositions that include Tantalum.

In [5]:
ta_indexes = []
for _ in range(len(data)):
    if ("Zr" in data['composition'][_] and "Ta" in data['composition'][_] and len(data['composition'][_])==5):
        ta_indexes.append(_)
    elif ("Zr" in data['composition'][_] and len(data['composition'][_])==4):
        ta_indexes.append(_) 
        
ta_dataframe = data[data.index.isin(ta_indexes)]
#display(ta_dataframe)        
        
x_values_exp = [_["Ta"] for _ in list(ta_dataframe['composition'])]
y_values_exp = list(ta_dataframe['Ionic Conductivity'])

In order to reduce noise in the neural network and deal with the inconsistencies in the data, we will filter repeated composition values from different measurements and replace the value for ionic conductivity with the median of the values. Similar approaches have been implemented in this [paper](https://iopscience.iop.org/article/10.1088/1361-651X/aaf8ca).

In [6]:
dup_indexes = data[data.duplicated(subset = data.columns.tolist()[0], keep=False)].index.tolist()

dup_dataframe =data[data.duplicated(subset = data.columns.tolist()[0], keep=False)]

duplicates = [[dup_dataframe.iloc[x][0], dup_dataframe.iloc[x][4], dup_indexes[x], dup_dataframe.iloc[x][-2]] for x in range(len(dup_dataframe.index))]
duplicate_compositions = {k: [] for k in set([dup_dataframe.iloc[x][0] for x in range(len(dup_dataframe.index))])}
duplicate_indexes = {k: [] for k in set([dup_dataframe.iloc[x][0] for x in range(len(dup_dataframe.index))])}
duplicate_years = {k: [] for k in set([dup_dataframe.iloc[x][0] for x in range(len(dup_dataframe.index))])}

for _ in duplicates:
    duplicate_compositions[_[0]].append(_[1])
    duplicate_indexes[_[0]].append(_[2]) 
    duplicate_years[_[0]].append(_[3])     

for k in duplicate_compositions:
    
    duplicate_compositions[k] = np.median(duplicate_compositions[k])
    data.at[duplicate_indexes[k][0], 'Ionic Conductivity'] = duplicate_compositions[k]  

    duplicate_years[k] = np.min(duplicate_years[k])
    data.at[duplicate_indexes[k][0], 'Year Published'] = duplicate_years[k]      
    data = data.drop(duplicate_indexes[k][1:], axis = 0)

data = data.reset_index()
data = data.drop(['index'], axis = 1)

After removing duplicates, we will query the dataset for the values that were substituted for the Tantalum compositions, the median of the experimental values.

In [7]:
ta_indexes = []
for _ in range(len(data)):
    if ("Zr" in data['composition'][_] and "Ta" in data['composition'][_] and len(data['composition'][_])==5):
        ta_indexes.append(_)
    elif ("Zr" in data['composition'][_] and len(data['composition'][_])==4):
        ta_indexes.append(_)        
        
ta_dataframe = data[data.index.isin(ta_indexes)]
#display(ta_dataframe)

x_values_dupmed = [_["Ta"] for _ in list(ta_dataframe['composition'])]
y_values_dupmed = list(ta_dataframe['Ionic Conductivity'])

sort_dupmed = list(zip(x_values_dupmed, y_values_dupmed))
sort_dupmed = sorted(sort_dupmed, key = lambda t: t[0])

x_values_dupmed = [item[0] for item in sort_dupmed ] 
y_values_dupmed = [item[1] for item in sort_dupmed ] 

The next cell produces a breakdown of the number of elements in the oxides compositions and a distribution of the elements present in the dataset.

In [8]:
import collections

freq = data["composition"]
list_freq = []

for _ in freq:
    a = [str(x) for x in _]
    list_freq.append(a)
    
list_freq_flat = [item for sublist in list_freq for item in sublist]  
listfreqctr = collections.Counter(list_freq_flat)
print(listfreqctr)
    
lengths = list(map(len,list_freq))
lenctr = collections.Counter(lengths)

print(lenctr)
# print(type(freq[0]))
# print(freq[0])
# print(list(freq[0])[0])

# print(type(list(freq[0])[0]))
# print(str(list(freq[0])[0]))

Counter({'Li': 100, 'La': 100, 'O': 100, 'Zr': 70, 'Nb': 40, 'Ta': 29, 'Ga': 16, 'Y': 15, 'Sr': 11, 'Ba': 10, 'Gd': 10, 'Ca': 7, 'Bi': 6, 'Al': 5, 'Sc': 3, 'Nd': 3, 'Sb': 2})
Counter({5: 62, 6: 31, 4: 6, 7: 1})


---

## 2. Matminer Descriptors

In [9]:
f =  MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"), cf.ValenceOrbital(props=['avg']), cf.ElementFraction()]) # Featurizers

X = np.array(f.featurize_many(data['composition'], ignore_errors=True)) # Array to store such features

measuring_temp_array = np.array(data['Measuring Temperature']).reshape(-1,1) # Here we are stacking the Measuring temperature numpy array into the features previously calculated to add it as a descriptor. 
X = np.hstack((X,measuring_temp_array))

y = data[property_interest].values # Separate the value we want to predict to use as labels.
years = data["Year Published"].values

# This code is to drop columns with std = 0. 
x_df = pd.DataFrame(X)
x_df = x_df.loc[:, x_df.std() != 0]
print(x_df.shape) # This shape is (#Entries, #Descriptors per entry)

# This code is to drop columns with std = 0. 
x_df_prior = pd.DataFrame(X)


(100, 105)


--- 

## 3. Regression Models (Architecture 1)

We will start by creating a models for regression with all these entries and descriptors.

### Neural Networks (90-60-30-1)


We set the architecture of the sequential feed-forward neural network we'll test. Weights are initialized with a Random Normal distribution and biases are initialized at zero.

We'll use this training function with a validation mean absolute error (mae) stopping function to train the model. A 10% validation set is set to be taken from the training.
<br>
A figure of training mae vs validation mae is shown. Overfitting occurs when the validation mae starts to increase, so we revert the weights to those of the best epoch.

In [10]:
from sklearn.utils import shuffle

# EXTRACTION OF THE DATA FOR THE DESCRIPTORS

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype = float) 
all_labels = y.copy()

# SPLIT INDICATION FOR TRAIN/TEST SETS
train_percent = 0.90
index_split_at = int (train_percent * len(all_labels))


# EARLY STOPPING CRITERIA
#mae_es= keras.callbacks.EarlyStopping(monitor='mean_squared_error', min_delta=1e-8, patience=200, verbose=1, mode='auto', restore_best_weights=True)
valmae_es= keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', min_delta=1e-10, patience=1000, verbose=0, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" %logs.get('loss') + '                                       \r') # Updates current Epoch Number

EPOCHS = 10000 # Number of EPOCHS

# NETWORK INITIALIZERS

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()
optimizer = tf.train.AdamOptimizer()

mae_st = []

for _i in range(10):
    print("Split", _i)

    # DATA SPLIT AND NORMALIZATION
    all_values, all_labels = shuffle(all_values, all_labels, random_state=0)

    train_values, test_values = np.split(all_values, [index_split_at])
    train_labels, test_labels = np.split(all_labels, [index_split_at])

    feature_mean = np.mean(train_values, axis=0)
    feature_std = np.std(train_values, axis=0)

    train_values = (train_values - feature_mean)/ (feature_std)
    test_values = (test_values - feature_mean)/ (feature_std)

    # NETWORK ARCHITECTURE

    neuralnetwork_model = Sequential()
    neuralnetwork_model.add(Dense(90, activation='relu', use_bias = True, input_shape=(train_values.shape[1], ), kernel_initializer=kernel_init, bias_initializer=bias_init))
    neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))
    neuralnetwork_model.add(Dense(30, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))
    neuralnetwork_model.add(Dense(1, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))

    neuralnetwork_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

    history = neuralnetwork_model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose = False, callbacks=[PrintEpNum(), valmae_es]) #  

    [loss, mae] = neuralnetwork_model.evaluate(test_values, test_labels, verbose=0)


    print("\n Test Loss: ", loss)
    mae_st.append(mae)

print("Mean AVG MAE - 10 Splits: %f"%(np.mean(mae_st)))

Split 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Current Epoch: 3282 Training Loss: 0.191564                                       
 Test Loss:  1.2104456424713135
Split 1
Current Epoch: 1049 Training Loss: 0.273764                                       
 Test Loss:  1.5149542093276978
Split 2
Current Epoch: 1049 Training Loss: 0.166175                                       
 Test Loss:  1.3422971963882446
Split 3
Current Epoch: 1038 Training Loss: 0.144613                                       
 Test Loss:  4.111766815185547
Split 4
Current Epoch: 1145 Training Loss: 0.167278                                       
 Test Loss:  0.6627296805381775
Split 5
Current Epoch: 1055 Training Loss: 0.246246                                       
 Test Loss:  0.6004053354263306
Split 6
Current Epoch: 1364 Training Loss: 0.229622                                       
 Test Loss:  0.5010675191879272
Split 7
Current Epoc

As a form of regularization, we will be exploring three options: Dropout, and the L1/L2 Layer weight regularizers.

### 3.1 Dropout

In [11]:
from sklearn.utils import shuffle

# EXTRACTION OF THE DATA FOR THE DESCRIPTORS

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype = float) 
all_labels = y.copy()

# SPLIT INDICATION FOR TRAIN/TEST SETS
train_percent = 0.90
index_split_at = int (train_percent * len(all_labels))


# EARLY STOPPING CRITERIA
#mae_es= keras.callbacks.EarlyStopping(monitor='mean_squared_error', min_delta=1e-8, patience=200, verbose=1, mode='auto', restore_best_weights=True)
valmae_es= keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', min_delta=1e-10, patience=1000, verbose=0, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" %logs.get('loss') + '                                       \r') # Updates current Epoch Number

EPOCHS = 10000 # Number of EPOCHS

# NETWORK INITIALIZERS

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()
optimizer = tf.train.AdamOptimizer()

mae_st = []


for drop in [0.1, 0.2,0.3]: 
    print("-----------------------------------------------")
    print("Testing %d%% Dropout "%(drop*100))
    
    for _i in range(10):
        print("Split", _i)

        # DATA SPLIT AND NORMALIZATION
        all_values, all_labels = shuffle(all_values, all_labels, random_state=0)

        train_values, test_values = np.split(all_values, [index_split_at])
        train_labels, test_labels = np.split(all_labels, [index_split_at])

        feature_mean = np.mean(train_values, axis=0)
        feature_std = np.std(train_values, axis=0)

        train_values = (train_values - feature_mean)/ (feature_std)
        test_values = (test_values - feature_mean)/ (feature_std)

        # NETWORK ARCHITECTURE

        neuralnetwork_model = Sequential()
        neuralnetwork_model.add(Dense(90, activation='relu', use_bias = True, input_shape=(train_values.shape[1], ), kernel_initializer=kernel_init, bias_initializer=bias_init))
        neuralnetwork_model.add(Dropout(drop))
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))
        neuralnetwork_model.add(Dropout(drop))
        neuralnetwork_model.add(Dense(30, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))
        neuralnetwork_model.add(Dropout(drop))
        neuralnetwork_model.add(Dense(1, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))

        neuralnetwork_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

        history = neuralnetwork_model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose = False, callbacks=[PrintEpNum(), valmae_es]) #  

        [loss, mae] = neuralnetwork_model.evaluate(test_values, test_labels, verbose=0)
        
        
        print("\n Test Loss: ", loss)
        mae_st.append(mae)

    print("Mean AVG MAE - 10 Splits - %d%% Dropout: %f"%(drop*100, np.mean(mae_st)))

-----------------------------------------------
Testing 10% Dropout 
Split 0
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Current Epoch: 3150 Training Loss: 0.375278                                       
 Test Loss:  1.0965170860290527
Split 1
Current Epoch: 1363 Training Loss: 0.439426                                       
 Test Loss:  1.3251070976257324
Split 2
Current Epoch: 1071 Training Loss: 0.396858                                       
 Test Loss:  1.0273689031600952
Split 3
Current Epoch: 1065 Training Loss: 0.426560                                       
 Test Loss:  3.0503926277160645
Split 4
Current Epoch: 1205 Training Loss: 0.426909                                       
 Test Loss:  0.5919569730758667
Split 5
Current Epoch: 1042 Training Loss: 0.616624                                       
 Test Loss:  0.48775404691696167
Split 6
Current Epoch: 1365 Training Loss: 0.451308                          

## 3.2 - L2 Regularization

In [12]:
from sklearn.utils import shuffle

# EXTRACTION OF THE DATA FOR THE DESCRIPTORS

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype = float) 
all_labels = y.copy()

# SPLIT INDICATION FOR TRAIN/TEST SETS
train_percent = 0.90
index_split_at = int (train_percent * len(all_labels))


# EARLY STOPPING CRITERIA
#mae_es= keras.callbacks.EarlyStopping(monitor='mean_squared_error', min_delta=1e-8, patience=200, verbose=1, mode='auto', restore_best_weights=True)
valmae_es= keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', min_delta=1e-10, patience=1000, verbose=0, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" %logs.get('loss') + '                                       \r') # Updates current Epoch Number

EPOCHS = 10000 # Number of EPOCHS

# NETWORK INITIALIZERS

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()
optimizer = tf.train.AdamOptimizer()

mae_st = []


for reg in [1e-4, 1e-5, 1e-6]: 
    print("-----------------------------------------------")
    print("Testing L2 Regularization: Penalty %f "%(reg))
    
    for _i in range(10):
        print("Split", _i)

        # DATA SPLIT AND NORMALIZATION
        all_values, all_labels = shuffle(all_values, all_labels, random_state=0)

        train_values, test_values = np.split(all_values, [index_split_at])
        train_labels, test_labels = np.split(all_labels, [index_split_at])

        feature_mean = np.mean(train_values, axis=0)
        feature_std = np.std(train_values, axis=0)

        train_values = (train_values - feature_mean)/ (feature_std)
        test_values = (test_values - feature_mean)/ (feature_std)

        # NETWORK ARCHITECTURE

        neuralnetwork_model = Sequential()
        neuralnetwork_model.add(Dense(90, activation='relu', use_bias = True, input_shape=(train_values.shape[1], ), kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l2(reg)))
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l2(reg) ))
        neuralnetwork_model.add(Dense(30, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l2(reg) ))
        neuralnetwork_model.add(Dense(1, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l2(reg) ))

        neuralnetwork_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

        history = neuralnetwork_model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose = False, callbacks=[PrintEpNum(), valmae_es]) #  

        [loss, mae] = neuralnetwork_model.evaluate(test_values, test_labels, verbose=0)
        
        
        print("\n Test Loss: ", loss)
        mae_st.append(mae)

    print("Mean AVG MAE - 10 Splits - L2 Regularization Penalty %f: %f "%(reg, np.mean(mae_st)))

-----------------------------------------------
Testing L2 Regularization: Penalty 0.000100 
Split 0
Current Epoch: 1741 Training Loss: 0.285532                                       
 Test Loss:  0.9004488587379456
Split 1
Current Epoch: 1057 Training Loss: 0.317207                                       
 Test Loss:  1.3330650329589844
Split 2
Current Epoch: 1051 Training Loss: 0.178800                                       
 Test Loss:  1.134377121925354
Split 3
Current Epoch: 1030 Training Loss: 0.183775                                       
 Test Loss:  4.5537543296813965
Split 4
Current Epoch: 1144 Training Loss: 0.186767                                       
 Test Loss:  0.7719566226005554
Split 5
Current Epoch: 1040 Training Loss: 0.282293                                       
 Test Loss:  0.6279588937759399
Split 6
Current Epoch: 1072 Training Loss: 0.298602                                       
 Test Loss:  0.4758690893650055
Split 7
Current Epoch: 1763 Training Loss: 0.23

## 3.3 - L1 Regularization

In [13]:
from sklearn.utils import shuffle

# EXTRACTION OF THE DATA FOR THE DESCRIPTORS

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype = float) 
all_labels = y.copy()

# SPLIT INDICATION FOR TRAIN/TEST SETS
train_percent = 0.90
index_split_at = int (train_percent * len(all_labels))


# EARLY STOPPING CRITERIA
#mae_es= keras.callbacks.EarlyStopping(monitor='mean_squared_error', min_delta=1e-8, patience=200, verbose=1, mode='auto', restore_best_weights=True)
valmae_es= keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', min_delta=1e-10, patience=1000, verbose=0, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" %logs.get('loss') + '                                       \r') # Updates current Epoch Number

EPOCHS = 10000 # Number of EPOCHS

# NETWORK INITIALIZERS

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()
optimizer = tf.train.AdamOptimizer()

mae_st = []


for reg in [1e-4, 1e-5, 1e-6]: 
    print("-----------------------------------------------")
    print("Testing L1 Regularization: Penalty %f "%(reg))
    
    for _i in range(10):
        print("Split", _i)

        # DATA SPLIT AND NORMALIZATION
        all_values, all_labels = shuffle(all_values, all_labels, random_state=0)

        train_values, test_values = np.split(all_values, [index_split_at])
        train_labels, test_labels = np.split(all_labels, [index_split_at])

        feature_mean = np.mean(train_values, axis=0)
        feature_std = np.std(train_values, axis=0)

        train_values = (train_values - feature_mean)/ (feature_std)
        test_values = (test_values - feature_mean)/ (feature_std)

        # NETWORK ARCHITECTURE

        neuralnetwork_model = Sequential()
        neuralnetwork_model.add(Dense(90, activation='relu', use_bias = True, input_shape=(train_values.shape[1], ), kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l1(reg)))
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l1(reg) ))
        neuralnetwork_model.add(Dense(30, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l1(reg) ))
        neuralnetwork_model.add(Dense(1, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l1(reg) ))

        neuralnetwork_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

        history = neuralnetwork_model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose = False, callbacks=[PrintEpNum(), valmae_es]) #  

        [loss, mae] = neuralnetwork_model.evaluate(test_values, test_labels, verbose=0)
        
        
        print("\n Test Loss: ", loss)
        mae_st.append(mae)

    print("Mean AVG MAE - 10 Splits - L1 Regularization Penalty %f: %f "%(reg, np.mean(mae_st)))

-----------------------------------------------
Testing L1 Regularization: Penalty 0.000100 
Split 0
Current Epoch: 1418 Training Loss: 0.375368                                       
 Test Loss:  0.9323380589485168
Split 1
Current Epoch: 1055 Training Loss: 0.413693                                       
 Test Loss:  1.5985743999481201
Split 2
Current Epoch: 1045 Training Loss: 0.225271                                       
 Test Loss:  1.343571662902832
Split 3
Current Epoch: 1042 Training Loss: 0.242513                                       
 Test Loss:  4.450512886047363
Split 4
Current Epoch: 1342 Training Loss: 0.306701                                       
 Test Loss:  0.7638652920722961
Split 5
Current Epoch: 1046 Training Loss: 0.346952                                       
 Test Loss:  0.6441807746887207
Split 6
Current Epoch: 1194 Training Loss: 0.376619                                       
 Test Loss:  0.6353306174278259
Split 7
Current Epoch: 2679 Training Loss: 0.267

--- 

## 4. Regression Models (Architecture 2)

We will start by creating a models for regression with all these entries and descriptors.

### Neural Networks (60-120-60-1)


We set the architecture of the sequential feed-forward neural network we'll test. Weights are initialized with a Random Normal distribution and biases are initialized at zero.

We'll use this training function with a validation mean absolute error (mae) stopping function to train the model. A 10% validation set is set to be taken from the training.
<br>
A figure of training mae vs validation mae is shown. Overfitting occurs when the validation mae starts to increase, so we revert the weights to those of the best epoch.

In [14]:
from sklearn.utils import shuffle

# EXTRACTION OF THE DATA FOR THE DESCRIPTORS

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype = float) 
all_labels = y.copy()

# SPLIT INDICATION FOR TRAIN/TEST SETS
train_percent = 0.90
index_split_at = int (train_percent * len(all_labels))


# EARLY STOPPING CRITERIA
#mae_es= keras.callbacks.EarlyStopping(monitor='mean_squared_error', min_delta=1e-8, patience=200, verbose=1, mode='auto', restore_best_weights=True)
valmae_es= keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', min_delta=1e-10, patience=1000, verbose=0, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" %logs.get('loss') + '                                       \r') # Updates current Epoch Number

EPOCHS = 10000 # Number of EPOCHS

# NETWORK INITIALIZERS

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()
optimizer = tf.train.AdamOptimizer()

mae_st = []


for _i in range(10):


    # DATA SPLIT AND NORMALIZATION
    all_values, all_labels = shuffle(all_values, all_labels, random_state=0)

    train_values, test_values = np.split(all_values, [index_split_at])
    train_labels, test_labels = np.split(all_labels, [index_split_at])

    feature_mean = np.mean(train_values, axis=0)
    feature_std = np.std(train_values, axis=0)

    train_values = (train_values - feature_mean)/ (feature_std)
    test_values = (test_values - feature_mean)/ (feature_std)

    # NETWORK ARCHITECTURE
    
    neuralnetwork_model = Sequential()
    neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, input_shape=(train_values.shape[1], ), kernel_initializer=kernel_init, bias_initializer=bias_init))
    neuralnetwork_model.add(Dense(120, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))
    neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))
    neuralnetwork_model.add(Dense(1, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))

    neuralnetwork_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

    history = neuralnetwork_model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose = False, callbacks=[PrintEpNum(), valmae_es]) #  

    [loss, mae] = neuralnetwork_model.evaluate(test_values, test_labels, verbose=0)

    mae_st.append(mae)
    
print("Mean AVG MAE - 10 Steps: ", np.mean(mae_st))

Mean AVG MAE - 10 Steps:  1.3464750319719314                                      


## 4.1 Dropout

In [15]:
from sklearn.utils import shuffle

# EXTRACTION OF THE DATA FOR THE DESCRIPTORS

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype = float) 
all_labels = y.copy()

# SPLIT INDICATION FOR TRAIN/TEST SETS
train_percent = 0.90
index_split_at = int (train_percent * len(all_labels))


# EARLY STOPPING CRITERIA
#mae_es= keras.callbacks.EarlyStopping(monitor='mean_squared_error', min_delta=1e-8, patience=200, verbose=1, mode='auto', restore_best_weights=True)
valmae_es= keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', min_delta=1e-10, patience=1000, verbose=0, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" %logs.get('loss') + '                                       \r') # Updates current Epoch Number

EPOCHS = 10000 # Number of EPOCHS

# NETWORK INITIALIZERS

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()
optimizer = tf.train.AdamOptimizer()

mae_st = []


for drop in [0.1, 0.2,0.3]: 
    print("-----------------------------------------------")
    print("Testing %d%% Dropout "%(drop*100))
    
    for _i in range(10):
        print("Split", _i)

        # DATA SPLIT AND NORMALIZATION
        all_values, all_labels = shuffle(all_values, all_labels, random_state=0)

        train_values, test_values = np.split(all_values, [index_split_at])
        train_labels, test_labels = np.split(all_labels, [index_split_at])

        feature_mean = np.mean(train_values, axis=0)
        feature_std = np.std(train_values, axis=0)

        train_values = (train_values - feature_mean)/ (feature_std)
        test_values = (test_values - feature_mean)/ (feature_std)

        # NETWORK ARCHITECTURE

        neuralnetwork_model = Sequential()
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, input_shape=(train_values.shape[1], ), kernel_initializer=kernel_init, bias_initializer=bias_init))
        neuralnetwork_model.add(Dropout(drop))
        neuralnetwork_model.add(Dense(120, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))
        neuralnetwork_model.add(Dropout(drop))
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))
        neuralnetwork_model.add(Dropout(drop))
        neuralnetwork_model.add(Dense(1, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init ))

        neuralnetwork_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

        history = neuralnetwork_model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose = False, callbacks=[PrintEpNum(), valmae_es]) #  

        [loss, mae] = neuralnetwork_model.evaluate(test_values, test_labels, verbose=0)
        
        
        print("\n Test Loss: ", loss)
        mae_st.append(mae)

    print("Mean AVG MAE - 10 Splits - %d%% Dropout: %f"%(drop*100, np.mean(mae_st)))

-----------------------------------------------
Testing 10% Dropout 
Split 0
Current Epoch: 1562 Training Loss: 0.446494                                       
 Test Loss:  0.746110200881958
Split 1
Current Epoch: 1078 Training Loss: 0.438903                                       
 Test Loss:  1.3682829141616821
Split 2
Current Epoch: 1047 Training Loss: 0.397131                                       
 Test Loss:  1.1333160400390625
Split 3
Current Epoch: 1065 Training Loss: 0.400298                                       
 Test Loss:  3.641481399536133
Split 4
Current Epoch: 1108 Training Loss: 0.406301                                       
 Test Loss:  0.7020031213760376
Split 5
Current Epoch: 1071 Training Loss: 0.405413                                       
 Test Loss:  0.6211980581283569
Split 6
Current Epoch: 1419 Training Loss: 0.413708                                       
 Test Loss:  0.5984992384910583
Split 7
Current Epoch: 1847 Training Loss: 0.475434                     

## 4.2 - L2 Regularization

In [16]:
from sklearn.utils import shuffle

# EXTRACTION OF THE DATA FOR THE DESCRIPTORS

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype = float) 
all_labels = y.copy()

# SPLIT INDICATION FOR TRAIN/TEST SETS
train_percent = 0.90
index_split_at = int (train_percent * len(all_labels))


# EARLY STOPPING CRITERIA
#mae_es= keras.callbacks.EarlyStopping(monitor='mean_squared_error', min_delta=1e-8, patience=200, verbose=1, mode='auto', restore_best_weights=True)
valmae_es= keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', min_delta=1e-10, patience=1000, verbose=0, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" %logs.get('loss') + '                                       \r') # Updates current Epoch Number

EPOCHS = 10000 # Number of EPOCHS

# NETWORK INITIALIZERS

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()
optimizer = tf.train.AdamOptimizer()

mae_st = []


for reg in [1e-4, 1e-5, 1e-6]: 
    print("-----------------------------------------------")
    print("Testing L2 Regularization: Penalty %f "%(reg))
    
    for _i in range(10):
        print("Split", _i)

        # DATA SPLIT AND NORMALIZATION
        all_values, all_labels = shuffle(all_values, all_labels, random_state=0)

        train_values, test_values = np.split(all_values, [index_split_at])
        train_labels, test_labels = np.split(all_labels, [index_split_at])

        feature_mean = np.mean(train_values, axis=0)
        feature_std = np.std(train_values, axis=0)

        train_values = (train_values - feature_mean)/ (feature_std)
        test_values = (test_values - feature_mean)/ (feature_std)

        # NETWORK ARCHITECTURE

        neuralnetwork_model = Sequential()
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, input_shape=(train_values.shape[1], ), kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l2(reg)))
        neuralnetwork_model.add(Dense(120, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l2(reg) ))
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l2(reg) ))
        neuralnetwork_model.add(Dense(1, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l2(reg) ))

        neuralnetwork_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

        history = neuralnetwork_model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose = False, callbacks=[PrintEpNum(), valmae_es]) #  

        [loss, mae] = neuralnetwork_model.evaluate(test_values, test_labels, verbose=0)
        
        
        print("\n Test Loss: ", loss)
        mae_st.append(mae)

    print("Mean AVG MAE - 10 Splits - L2 Regularization Penalty %f: %f "%(reg, np.mean(mae_st)))

-----------------------------------------------
Testing L2 Regularization: Penalty 0.000100 
Split 0
Current Epoch: 1582 Training Loss: 0.247738                                       
 Test Loss:  0.8544228076934814
Split 1
Current Epoch: 1201 Training Loss: 0.235488                                       
 Test Loss:  0.9531177878379822
Split 2
Current Epoch: 1059 Training Loss: 0.200203                                       
 Test Loss:  1.15052330493927
Split 3
Current Epoch: 1021 Training Loss: 0.143207                                       
 Test Loss:  4.000640392303467
Split 4
Current Epoch: 1101 Training Loss: 0.162880                                       
 Test Loss:  0.5941249132156372
Split 5
Current Epoch: 1080 Training Loss: 0.274732                                       
 Test Loss:  0.6986626982688904
Split 6
Current Epoch: 1063 Training Loss: 0.227749                                       
 Test Loss:  0.5755887627601624
Split 7
Current Epoch: 1386 Training Loss: 0.2447

## 4.3 - L1 Regularization

In [17]:
from sklearn.utils import shuffle

# EXTRACTION OF THE DATA FOR THE DESCRIPTORS

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype = float) 
all_labels = y.copy()

# SPLIT INDICATION FOR TRAIN/TEST SETS
train_percent = 0.90
index_split_at = int (train_percent * len(all_labels))


# EARLY STOPPING CRITERIA
#mae_es= keras.callbacks.EarlyStopping(monitor='mean_squared_error', min_delta=1e-8, patience=200, verbose=1, mode='auto', restore_best_weights=True)
valmae_es= keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', min_delta=1e-10, patience=1000, verbose=0, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" %logs.get('loss') + '                                       \r') # Updates current Epoch Number

EPOCHS = 10000 # Number of EPOCHS

# NETWORK INITIALIZERS

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()
optimizer = tf.train.AdamOptimizer()

mae_st = []


for reg in [1e-4, 1e-5, 1e-6]: 
    print("-----------------------------------------------")
    print("Testing L1 Regularization: Penalty %f "%(reg))
    
    for _i in range(10):
        print("Split", _i)

        # DATA SPLIT AND NORMALIZATION
        all_values, all_labels = shuffle(all_values, all_labels, random_state=0)

        train_values, test_values = np.split(all_values, [index_split_at])
        train_labels, test_labels = np.split(all_labels, [index_split_at])

        feature_mean = np.mean(train_values, axis=0)
        feature_std = np.std(train_values, axis=0)

        train_values = (train_values - feature_mean)/ (feature_std)
        test_values = (test_values - feature_mean)/ (feature_std)

        # NETWORK ARCHITECTURE

        neuralnetwork_model = Sequential()
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, input_shape=(train_values.shape[1], ), kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l1(reg)))
        neuralnetwork_model.add(Dense(120, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l1(reg) ))
        neuralnetwork_model.add(Dense(60, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l1(reg) ))
        neuralnetwork_model.add(Dense(1, activation='relu', use_bias = True, kernel_initializer=kernel_init, bias_initializer=bias_init, kernel_regularizer=regularizers.l1(reg) ))

        neuralnetwork_model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

        history = neuralnetwork_model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose = False, callbacks=[PrintEpNum(), valmae_es]) #  

        [loss, mae] = neuralnetwork_model.evaluate(test_values, test_labels, verbose=0)
        
        
        print("\n Test Loss: ", loss)
        mae_st.append(mae)

    print("Mean AVG MAE - 10 Splits - L1 Regularization Penalty %f: %f "%(reg, np.mean(mae_st)))

-----------------------------------------------
Testing L1 Regularization: Penalty 0.000100 
Split 0
Current Epoch: 1927 Training Loss: 0.329331                                       
 Test Loss:  1.0051826238632202
Split 1
Current Epoch: 1082 Training Loss: 0.313305                                       
 Test Loss:  1.359147071838379
Split 2
Current Epoch: 1081 Training Loss: 0.227364                                       
 Test Loss:  1.2636651992797852
Split 3
Current Epoch: 1024 Training Loss: 0.249700                                       
 Test Loss:  4.04002571105957
Split 4
Current Epoch: 1112 Training Loss: 0.277568                                       
 Test Loss:  0.852234423160553
Split 5
Current Epoch: 1032 Training Loss: 0.378093                                       
 Test Loss:  0.653967022895813
Split 6
Current Epoch: 1229 Training Loss: 0.319922                                       
 Test Loss:  0.5501052737236023
Split 7
Current Epoch: 1104 Training Loss: 0.339861